In [6]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
import os

# ==============================================================================
# 0. KONFIGURACJA ŚCIEŻEK I PARAMETRÓW
# ==============================================================================

# Ścieżka bazowa, gdzie znajduje się folder PROJEKT i pliki audio (np. MIX/M0055.wav)

BASE_PATH = 'C:/Users/macie/Desktop/PROJEKT/' 

# Pełna ścieżka do pliku CSV (train.csv musi znajdować się bezpośrednio w folderze PROJEKT)
CSV_FILE_FULL_PATH = os.path.join(BASE_PATH, 'train.csv') 

# Parametry Filtra Bandpass (DO ZMIANY PO ANALIZIE FFT!)
F_LOW = 100.0  
F_HIGH = 1500.0 
FILTER_ORDER = 4 

# ==============================================================================
# 1. NARZĘDZIA DSP (Filtry)
# ==============================================================================

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    """Implementacja filtra pasmowoprzepustowego Butterwortha."""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y_filtered = lfilter(b, a, data)
    return y_filtered

# ==============================================================================
# 2. ANALIZA I WIZUALIZACJA
# ==============================================================================

def analyze_and_plot_signal(y, fs, title_prefix, filename):
    """Wykonuje Analizę Częstotliwościową (FFT) i wizualizację (Spektrogramy)."""
    
    # --- A. ANALIZA CZĘSTOTLIWOŚCIOWA (FFT) ---
    N = len(y)
    Y = np.fft.fft(y)
    Y_abs = np.abs(Y) / N
    f = np.fft.fftfreq(N, 1/fs)
    jednostronne_index = np.where(f >= 0)
    f_jednostronne = f[jednostronne_index]
    Y_jednostronne = Y_abs[jednostronne_index]
    Y_jednostronne[1:] = 2 * Y_jednostronne[1:]

    # Wizualizacja Widma Amplitudowego (FFT)
    plt.figure(figsize=(10, 5))
    plt.plot(f_jednostronne, Y_jednostronne)
    plt.title(f'{title_prefix}: Widmo Amplitudowe (FFT)')
    plt.xlabel('Częstotliwość (Hz)')
    plt.ylabel('Amplituda (Normalizowana)')
    plt.xlim(0, fs / 2)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{title_prefix}_FFT.png') 
    plt.close()

    # --- B. WIZUALIZACJA SPEKTROGRAMÓW (Surowy sygnał) ---
    S = librosa.stft(y)
    S_dB = librosa.amplitude_to_db(np.abs(S), ref=np.max)
    
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_dB, sr=fs, x_axis='time', y_axis='hz', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'{title_prefix}: Spektrogram SUROWY')
    plt.ylim(0, 2000) 
    plt.tight_layout()
    plt.savefig(f'{title_prefix}_SPEK_RAW.png') 
    plt.close()

    # --- C. ZASTOSOWANIE FILTRA I ANALIZA PASMOWA ---
    y_filtered = butter_bandpass_filter(y, F_LOW, F_HIGH, fs, FILTER_ORDER)
    S_filtered = librosa.stft(y_filtered)
    S_dB_filtered = librosa.amplitude_to_db(np.abs(S_filtered), ref=np.max)
    
    # Wizualizacja Spektrogramu FILTROWANEGO
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_dB_filtered, sr=fs, x_axis='time', y_axis='hz', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'{title_prefix}: Spektrogram FILTROWANY ({F_LOW}-{F_HIGH} Hz)')
    plt.ylim(0, 2000)
    plt.tight_layout()
    plt.savefig(f'{title_prefix}_SPEK_FILTERED.png') 
    plt.close()

# ==============================================================================
# 3. GŁÓWNA PĘTLA
# ==============================================================================

# Wczytanie CSV z pełnej ścieżki
try:
    df_train = pd.read_csv(CSV_FILE_FULL_PATH) 
    print(f"Sukces: Wczytano {len(df_train)} rekordów z {CSV_FILE_FULL_PATH}")
except FileNotFoundError:
    print(f"BŁĄD KRYTYCZNY: Nie znaleziono pliku CSV: {CSV_FILE_FULL_PATH}. Sprawdź, czy plik jest w folderze PROJEKT.")
    exit()

# Wybór po jednym reprezentatywnym pliku z każdej klasy
try:
    file_pathology = df_train[df_train['label'] == 'Pathology'].iloc[0]['filename']
    file_normal = df_train[df_train['label'] == 'Normal'].iloc[0]['filename']
except IndexError:
    print("Błąd: Nie znaleziono wystarczającej liczby próbek 'Normal' lub 'Pathology' w train.csv.")
    exit()

files_to_process = {
    'PATHOLOGY_EX': file_pathology,
    'NORMAL_EX': file_normal
}

print("--- Rozpoczęcie przetwarzania sygnałów ---")

for label, filename in files_to_process.items():
    # Pełna ścieżka do pliku audio
    full_path = os.path.join(BASE_PATH, filename)
    print(f"\nPrzetwarzanie pliku: {label} ({full_path})")

    try:
        # Wczytanie sygnału audio
        y, fs = librosa.load(full_path, sr=None)
        print(f"  Częstotliwość próbkowania (fs): {fs} Hz. Długość sygnału: {len(y)} próbek.")
        
        # Analiza i wizualizacja
        analyze_and_plot_signal(y, fs, label, filename)
        print(f"  Analiza zakończona. Wykresy zapisane jako: {label}_*.png")

    except FileNotFoundError:
        print(f"  BŁĄD: Nie znaleziono pliku audio: {full_path}. Sprawdź, czy ścieżka w pliku CSV ({filename}) jest poprawna względem {BASE_PATH}!")
    except Exception as e:
        print(f"  Wystąpił nieznany błąd podczas przetwarzania: {e}")

print("\n--- Przetwarzanie ZAKOŃCZONE ---")

Sukces: Wczytano 195 rekordów z C:/Users/macie/Desktop/PROJEKT/train.csv
--- Rozpoczęcie przetwarzania sygnałów ---

Przetwarzanie pliku: PATHOLOGY_EX (C:/Users/macie/Desktop/PROJEKT/MIX/M0055.wav)
  Częstotliwość próbkowania (fs): 4000 Hz. Długość sygnału: 60000 próbek.
  Analiza zakończona. Wykresy zapisane jako: PATHOLOGY_EX_*.png

Przetwarzanie pliku: NORMAL_EX (C:/Users/macie/Desktop/PROJEKT/HS/M_N_LLSB.wav)
  Częstotliwość próbkowania (fs): 4000 Hz. Długość sygnału: 60000 próbek.
  Analiza zakończona. Wykresy zapisane jako: NORMAL_EX_*.png

--- Przetwarzanie ZAKOŃCZONE ---


In [7]:
import os
import sys


print("Bieżący katalog roboczy (CWD):")
print(os.getcwd())


print("\nŚcieżka BASE_PATH:")
print('C:/Users/macie/Desktop/PROJEKT/')

Bieżący katalog roboczy (CWD):
C:\Users\macie

Ścieżka BASE_PATH:
C:/Users/macie/Desktop/PROJEKT/
